In [5]:
import re
import random
data_path = "human_text.txt"
data_path2 = "robot_text.txt"
# Defining lines as a list of each line
with open(data_path, 'r', encoding='utf-8') as f:
  lines = f.read().split('\n')
with open(data_path2, 'r', encoding='utf-8') as f:
  lines2 = f.read().split('\n')
lines = [re.sub(r"\[\w+\]",'hi',line) for line in lines]
lines = [" ".join(re.findall(r"\w+",line)) for line in lines]
lines2 = [re.sub(r"\[\w+\]",'',line) for line in lines2]
lines2 = [" ".join(re.findall(r"\w+",line)) for line in lines2]
# Grouping lines by response pair
pairs = list(zip(lines,lines2))
#random.shuffle(pairs)

In [6]:
import numpy as np

input_docs = []
target_docs = []
input_tokens = set()
target_tokens = set()
for line in pairs[:400]:
  input_doc, target_doc = line[0], line[1]
  # Appending each input sentence to input_docs
  input_docs.append(input_doc)
  # Splitting words from punctuation  
  target_doc = " ".join(re.findall(r"[\w']+|[^\s\w]", target_doc))
  # Redefine target_doc below and append it to target_docs
  target_doc = '<START> ' + target_doc + ' <END>'
  target_docs.append(target_doc)
  
  # Now we split up each sentence into words and add each unique word to our vocabulary set
  for token in re.findall(r"[\w']+|[^\s\w]", input_doc):
    if token not in input_tokens:
      input_tokens.add(token)
  for token in target_doc.split():
    if token not in target_tokens:
      target_tokens.add(token)
input_tokens = sorted(list(input_tokens))
target_tokens = sorted(list(target_tokens))
num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_tokens)

input_features_dict = dict(
    [(token, i) for i, token in enumerate(input_tokens)])
target_features_dict = dict(
    [(token, i) for i, token in enumerate(target_tokens)])

reverse_input_features_dict = dict(
    (i, token) for token, i in input_features_dict.items())
reverse_target_features_dict = dict(
    (i, token) for token, i in target_features_dict.items())


max_encoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", input_doc)) for input_doc in input_docs])
max_decoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", target_doc)) for target_doc in target_docs])

encoder_input_data = np.zeros(
    (len(input_docs), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for line, (input_doc, target_doc) in enumerate(zip(input_docs, target_docs)):
    for timestep, token in enumerate(re.findall(r"[\w']+|[^\s\w]", input_doc)):
        #Assign 1. for the current line, timestep, & word in encoder_input_data
        encoder_input_data[line, timestep, input_features_dict[token]] = 1.
    
    for timestep, token in enumerate(target_doc.split()):
        decoder_input_data[line, timestep, target_features_dict[token]] = 1.
        if timestep > 0:
            decoder_target_data[line, timestep - 1, target_features_dict[token]] = 1.

In [9]:
print(pairs[:5])
print(input_docs[:5])

[('hi', 'hi there how are you'), ('oh thanks i m fine this is an evening in my timezone', 'here is afternoon'), ('how do you feel today tell me something about yourself', 'my name is rdany but you can call me dany the r means robot i hope we can be virtual friends'), ('how many virtual friends have you got', 'i have many but not enough to fully understand humans beings'), ('is that forbidden for you to tell the exact number', 'i ve talked with 143 users counting 7294 lines of text')]
['hi', 'oh thanks i m fine this is an evening in my timezone', 'how do you feel today tell me something about yourself', 'how many virtual friends have you got', 'is that forbidden for you to tell the exact number']


In [10]:
from tensorflow import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model
#Dimensionality
dimensionality = 256
#The batch size and number of epochs
batch_size = 10
epochs = 600
#Encoder
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = LSTM(dimensionality, return_state=True)
encoder_outputs, state_hidden, state_cell = encoder_lstm(encoder_inputs)
encoder_states = [state_hidden, state_cell]
#Decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(dimensionality, return_sequences=True, return_state=True)
decoder_outputs, decoder_state_hidden, decoder_state_cell = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

#Model
training_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
#Compiling
training_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'], sample_weight_mode='temporal')
#Training
training_model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size = batch_size, epochs = epochs, validation_split = 0.2)
training_model.save('training_model.h5')

Epoch 1/600
32/32 [==============================] - 26s 809ms/step - loss: 1.2238 - accuracy: 0.0222 - val_loss: 1.3509 - val_accuracy: 0.0200
Epoch 2/600
32/32 [==============================] - 25s 789ms/step - loss: 1.1383 - accuracy: 0.0241 - val_loss: 1.3870 - val_accuracy: 0.0200
Epoch 3/600
32/32 [==============================] - 25s 776ms/step - loss: 1.1406 - accuracy: 0.0236 - val_loss: 1.4530 - val_accuracy: 0.0200
Epoch 4/600
32/32 [==============================] - 25s 769ms/step - loss: 1.1094 - accuracy: 0.0244 - val_loss: 1.4616 - val_accuracy: 0.0207
Epoch 5/600
32/32 [==============================] - 26s 797ms/step - loss: 1.0973 - accuracy: 0.0241 - val_loss: 1.4811 - val_accuracy: 0.0203
Epoch 6/600
32/32 [==============================] - 24s 759ms/step - loss: 1.0903 - accuracy: 0.0246 - val_loss: 1.4791 - val_accuracy: 0.0205
Epoch 7/600
32/32 [==============================] - 26s 801ms/step - loss: 1.0753 - accuracy: 0.0239 - val_loss: 1.4773 - val_accuracy:

32/32 [==============================] - 24s 756ms/step - loss: 0.3232 - accuracy: 0.1468 - val_loss: 1.7420 - val_accuracy: 0.0220
Epoch 58/600
32/32 [==============================] - 25s 770ms/step - loss: 0.3156 - accuracy: 0.1486 - val_loss: 1.7747 - val_accuracy: 0.0215
Epoch 59/600
32/32 [==============================] - 25s 781ms/step - loss: 0.3018 - accuracy: 0.1511 - val_loss: 1.7483 - val_accuracy: 0.0235
Epoch 60/600
32/32 [==============================] - 24s 765ms/step - loss: 0.2880 - accuracy: 0.1544 - val_loss: 1.7852 - val_accuracy: 0.0225
Epoch 61/600
32/32 [==============================] - 26s 818ms/step - loss: 0.2777 - accuracy: 0.1558 - val_loss: 1.7674 - val_accuracy: 0.0203
Epoch 62/600
32/32 [==============================] - 26s 814ms/step - loss: 0.2701 - accuracy: 0.1577 - val_loss: 1.7727 - val_accuracy: 0.0217
Epoch 63/600
32/32 [==============================] - 27s 831ms/step - loss: 0.2598 - accuracy: 0.1602 - val_loss: 1.7787 - val_accuracy: 0.023

32/32 [==============================] - 17s 536ms/step - loss: 0.0926 - accuracy: 0.1849 - val_loss: 2.0031 - val_accuracy: 0.0207
Epoch 114/600
32/32 [==============================] - 17s 537ms/step - loss: 0.0915 - accuracy: 0.1849 - val_loss: 2.0365 - val_accuracy: 0.0195
Epoch 115/600
32/32 [==============================] - 17s 540ms/step - loss: 0.0910 - accuracy: 0.1844 - val_loss: 2.0188 - val_accuracy: 0.0182
Epoch 116/600
32/32 [==============================] - 17s 537ms/step - loss: 0.0926 - accuracy: 0.1846 - val_loss: 2.0185 - val_accuracy: 0.0200
Epoch 117/600
32/32 [==============================] - 17s 536ms/step - loss: 0.0866 - accuracy: 0.1859 - val_loss: 2.0324 - val_accuracy: 0.0192
Epoch 118/600
32/32 [==============================] - 17s 537ms/step - loss: 0.0859 - accuracy: 0.1863 - val_loss: 2.0445 - val_accuracy: 0.0198
Epoch 119/600
32/32 [==============================] - 17s 539ms/step - loss: 0.0846 - accuracy: 0.1854 - val_loss: 2.0357 - val_accuracy:

32/32 [==============================] - 22s 673ms/step - loss: 0.0612 - accuracy: 0.1904 - val_loss: 2.1807 - val_accuracy: 0.0185
Epoch 170/600
32/32 [==============================] - 20s 637ms/step - loss: 0.0590 - accuracy: 0.1912 - val_loss: 2.1814 - val_accuracy: 0.0185
Epoch 171/600
32/32 [==============================] - 18s 577ms/step - loss: 0.0576 - accuracy: 0.1917 - val_loss: 2.1699 - val_accuracy: 0.0178
Epoch 172/600
32/32 [==============================] - 16s 513ms/step - loss: 0.0564 - accuracy: 0.1919 - val_loss: 2.1776 - val_accuracy: 0.0180
Epoch 173/600
32/32 [==============================] - 20s 611ms/step - loss: 0.0560 - accuracy: 0.1921 - val_loss: 2.1821 - val_accuracy: 0.0188
Epoch 174/600
32/32 [==============================] - 20s 628ms/step - loss: 0.0562 - accuracy: 0.1927 - val_loss: 2.1733 - val_accuracy: 0.0190
Epoch 175/600
32/32 [==============================] - 18s 569ms/step - loss: 0.0584 - accuracy: 0.1913 - val_loss: 2.1827 - val_accuracy:

32/32 [==============================] - 26s 817ms/step - loss: 0.0234 - accuracy: 0.2029 - val_loss: 2.2514 - val_accuracy: 0.0178
Epoch 226/600
32/32 [==============================] - 26s 813ms/step - loss: 0.0273 - accuracy: 0.2014 - val_loss: 2.2637 - val_accuracy: 0.0178
Epoch 227/600
32/32 [==============================] - 27s 840ms/step - loss: 0.0331 - accuracy: 0.2003 - val_loss: 2.2409 - val_accuracy: 0.0178
Epoch 228/600
32/32 [==============================] - 32s 1s/step - loss: 0.0258 - accuracy: 0.2013 - val_loss: 2.2379 - val_accuracy: 0.0195
Epoch 229/600
32/32 [==============================] - 28s 880ms/step - loss: 0.0300 - accuracy: 0.2001 - val_loss: 2.2629 - val_accuracy: 0.0178
Epoch 230/600
32/32 [==============================] - 28s 862ms/step - loss: 0.0295 - accuracy: 0.2010 - val_loss: 2.2645 - val_accuracy: 0.0168
Epoch 231/600
32/32 [==============================] - 27s 852ms/step - loss: 0.0269 - accuracy: 0.2018 - val_loss: 2.2479 - val_accuracy: 0.

...

32/32 [==============================] - 26s 825ms/step - loss: 0.0091 - accuracy: 0.2072 - val_loss: 2.3776 - val_accuracy: 0.0192
Epoch 338/600
32/32 [==============================] - 26s 818ms/step - loss: 0.0059 - accuracy: 0.2078 - val_loss: 2.3627 - val_accuracy: 0.0192
Epoch 339/600
32/32 [==============================] - 27s 841ms/step - loss: 0.0058 - accuracy: 0.2077 - val_loss: 2.3887 - val_accuracy: 0.0192
Epoch 340/600
32/32 [==============================] - 26s 813ms/step - loss: 0.0050 - accuracy: 0.2079 - val_loss: 2.3969 - val_accuracy: 0.0182
Epoch 341/600
32/32 [==============================] - 26s 827ms/step - loss: 0.0061 - accuracy: 0.2074 - val_loss: 2.4031 - val_accuracy: 0.0175
Epoch 342/600
32/32 [==============================] - 26s 817ms/step - loss: 0.0060 - accuracy: 0.2076 - val_loss: 2.3948 - val_accuracy: 0.0168
Epoch 343/600
32/32 [==============================] - 26s 818ms/step - loss: 0.0057 - accuracy: 0.2075 - val_loss: 2.3937 - val_accuracy:

32/32 [==============================] - 26s 808ms/step - loss: 0.0031 - accuracy: 0.2082 - val_loss: 2.4186 - val_accuracy: 0.0175
Epoch 394/600
32/32 [==============================] - 26s 820ms/step - loss: 0.0027 - accuracy: 0.2083 - val_loss: 2.4274 - val_accuracy: 0.0162
Epoch 395/600
32/32 [==============================] - 26s 810ms/step - loss: 0.0026 - accuracy: 0.2084 - val_loss: 2.4207 - val_accuracy: 0.0178
Epoch 396/600
32/32 [==============================] - 26s 809ms/step - loss: 0.0019 - accuracy: 0.2086 - val_loss: 2.4597 - val_accuracy: 0.0168
Epoch 397/600
32/32 [==============================] - 26s 819ms/step - loss: 0.0031 - accuracy: 0.2084 - val_loss: 2.4286 - val_accuracy: 0.0182
Epoch 398/600
32/32 [==============================] - 26s 803ms/step - loss: 0.0020 - accuracy: 0.2085 - val_loss: 2.4530 - val_accuracy: 0.0172
Epoch 399/600
32/32 [==============================] - 26s 822ms/step - loss: 0.0025 - accuracy: 0.2085 - val_loss: 2.4376 - val_accuracy:

32/32 [==============================] - 26s 813ms/step - loss: 0.0012 - accuracy: 0.2089 - val_loss: 2.4472 - val_accuracy: 0.0198
Epoch 450/600
32/32 [==============================] - 26s 819ms/step - loss: 0.0023 - accuracy: 0.2083 - val_loss: 2.4425 - val_accuracy: 0.0195
Epoch 451/600
32/32 [==============================] - 26s 811ms/step - loss: 0.0014 - accuracy: 0.2085 - val_loss: 2.4487 - val_accuracy: 0.0188
Epoch 452/600
32/32 [==============================] - 26s 819ms/step - loss: 0.0016 - accuracy: 0.2084 - val_loss: 2.4516 - val_accuracy: 0.0185
Epoch 453/600
32/32 [==============================] - 26s 817ms/step - loss: 0.0059 - accuracy: 0.2078 - val_loss: 2.4372 - val_accuracy: 0.0190
Epoch 454/600
32/32 [==============================] - 26s 823ms/step - loss: 0.0018 - accuracy: 0.2085 - val_loss: 2.4884 - val_accuracy: 0.0178
Epoch 455/600
32/32 [==============================] - 26s 826ms/step - loss: 0.0016 - accuracy: 0.2086 - val_loss: 2.4822 - val_accuracy:

32/32 [==============================] - 26s 819ms/step - loss: 0.0030 - accuracy: 0.2087 - val_loss: 2.4675 - val_accuracy: 0.0178
Epoch 506/600
32/32 [==============================] - 26s 820ms/step - loss: 0.0014 - accuracy: 0.2087 - val_loss: 2.4709 - val_accuracy: 0.0188
Epoch 507/600
32/32 [==============================] - 27s 830ms/step - loss: 0.0012 - accuracy: 0.2087 - val_loss: 2.4797 - val_accuracy: 0.0192
Epoch 508/600
32/32 [==============================] - 26s 819ms/step - loss: 0.0026 - accuracy: 0.2086 - val_loss: 2.4911 - val_accuracy: 0.0180
Epoch 509/600
32/32 [==============================] - 27s 833ms/step - loss: 0.0043 - accuracy: 0.2081 - val_loss: 2.4848 - val_accuracy: 0.0182
Epoch 510/600
32/32 [==============================] - 26s 826ms/step - loss: 0.0030 - accuracy: 0.2085 - val_loss: 2.4843 - val_accuracy: 0.0190
Epoch 511/600
32/32 [==============================] - 26s 817ms/step - loss: 0.0028 - accuracy: 0.2084 - val_loss: 2.5200 - val_accuracy:

32/32 [==============================] - 31s 971ms/step - loss: 0.0014 - accuracy: 0.2086 - val_loss: 2.5002 - val_accuracy: 0.0180
Epoch 562/600
32/32 [==============================] - 31s 979ms/step - loss: 9.0136e-04 - accuracy: 0.2087 - val_loss: 2.5134 - val_accuracy: 0.0195
Epoch 563/600
32/32 [==============================] - 29s 919ms/step - loss: 9.6024e-04 - accuracy: 0.2087 - val_loss: 2.5067 - val_accuracy: 0.0180
Epoch 564/600
32/32 [==============================] - 31s 963ms/step - loss: 0.0019 - accuracy: 0.2084 - val_loss: 2.5014 - val_accuracy: 0.0200
Epoch 565/600
32/32 [==============================] - 29s 906ms/step - loss: 0.0028 - accuracy: 0.2084 - val_loss: 2.4978 - val_accuracy: 0.0188
Epoch 566/600
32/32 [==============================] - 27s 838ms/step - loss: 0.0041 - accuracy: 0.2081 - val_loss: 2.5050 - val_accuracy: 0.0175
Epoch 567/600
32/32 [==============================] - 26s 823ms/step - loss: 0.0041 - accuracy: 0.2082 - val_loss: 2.5004 - val_a

In [12]:
from keras.models import load_model
training_model = load_model('training_model.h5')
encoder_inputs = training_model.input[0]
encoder_outputs, state_h_enc, state_c_enc = training_model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

latent_dim = 256
decoder_state_input_hidden = Input(shape=(latent_dim,))
decoder_state_input_cell = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_hidden, decoder_state_input_cell]
decoder_outputs, state_hidden, state_cell = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_hidden, state_cell]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

def decode_response(test_input):
    #Getting the output states to pass into the decoder
    states_value = encoder_model.predict(test_input)
    #Generating empty target sequence of length 1
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    #Setting the first token of target sequence with the start token
    target_seq[0, 0, target_features_dict['<START>']] = 1.
    
    #A variable to store our response word by word
    decoded_sentence = ''
    
    stop_condition = False
    while not stop_condition:
        #Predicting output tokens with probabilities and states
        output_tokens, hidden_state, cell_state = decoder_model.predict([target_seq] + states_value)
#Choosing the one with highest probability
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_features_dict[sampled_token_index]
        decoded_sentence += " " + sampled_token
#Stop if hit max length or found the stop token
        if (sampled_token == '<END>' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True
#Update the target sequence
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.
        #Update states
        states_value = [hidden_state, cell_state]
    return decoded_sentence

In [ ]:
class ChatBot:
  negative_responses = ("no", "nope", "nah", "naw", "not a chance", "sorry")
  exit_commands = ("quit", "pause", "exit", "goodbye", "bye", "later", "stop")
#Method to start the conversation
  def start_chat(self):
    user_response = input("Hi, I'm a chatbot trained on random dialogs. Would you like to chat with me?\n")
    
    if user_response in self.negative_responses:
      print("Ok, have a great day!")
      return
    self.chat(user_response)
#Method to handle the conversation
  def chat(self, reply):
    while not self.make_exit(reply):
      reply = input(self.generate_response(reply)+"\n")
    
  #Method to convert user input into a matrix
  def string_to_matrix(self, user_input):
    tokens = re.findall(r"[\w']+|[^\s\w]", user_input)
    user_input_matrix = np.zeros(
      (1, max_encoder_seq_length, num_encoder_tokens),
      dtype='float32')
    for timestep, token in enumerate(tokens):
      if token in input_features_dict:
        user_input_matrix[0, timestep, input_features_dict[token]] = 1.
    return user_input_matrix
  
  #Method that will create a response using seq2seq model we built
  def generate_response(self, user_input):
    input_matrix = self.string_to_matrix(user_input)
    chatbot_response = decode_response(input_matrix)
    #Remove <START> and <END> tokens from chatbot_response
    chatbot_response = chatbot_response.replace("<START>",'')
    chatbot_response = chatbot_response.replace("<END>",'')
    return chatbot_response
#Method to check for exit commands
  def make_exit(self, reply):
    for exit_command in self.exit_commands:
      if exit_command in reply:
        print("Ok, have a great day!")
        return True
    return False
  
chatbot = ChatBot()
chatbot.start_chat()

Hi, I'm a chatbot trained on random dialogs. Would you like to chat with me?
yes
 let me know when you have it working 
I have it working
 let me think do you know ifttt maybe you can do something
Yup I can do something
 hi there how are you today 
Doing great, how about you
 yes are good friends i like that 
who's good friends?
 i m fine and you 
good


In [0]:
chatbot.start_chat()

Hi, I'm a chatbot trained on random dialogs. Would you like to chat with me?
hi there
 hi there how are you today 
i m fine
 everything is fine on this side 
nice
 how is work going today 
all is good
 i m trying to learn like with the ukulele 
okay bye
Ok, have a great day!
